# 1. Introduction

Nama : Muhammad Asril Hanif

Batch : RMT-034

Notebook ini dibuat untuk melakukan validasi pada data yang telah melalui proses DAG Apache Airflow

# Libraries

In [1]:
# Libraries great expectations
from great_expectations.data_context import FileDataContext

# 3. Data Loading

In [2]:
# Define context
context = FileDataContext.create(project_root_dir='./')

Mendefinisikan context untuk proses validasi

In [6]:
# Define Datasource name
datasource_name = 'table-m3'
datasource = context.sources.add_pandas(datasource_name)

# Memberi nama pada aset data
asset_name = 'data'
path_to_data = 'p2m3_asril_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Membuat batch request
batch_request = asset.build_batch_request()

Mendefinisikan nama datasource, path data, dan membuat batch request untuk prasyarat melakukan proses validasi

# 4. Great Expectation

In [7]:
# Create an expectation suite
expectation_suite_name = 'expectation-m3-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,brand_id,brand_name,country,year,sustainability_rating,material_type,eco_friendly_manufacturing,carbon_footprint_mt,water_usage_liters,waste_production_kg,recycling_programs,product_lines,average_price_usd,market_trend,certifications
0,BRAND-0001,Brand_1,Australia,2018,D,Tencel,False,1.75,4511152.79,97844.11,False,2,38.33,Growing,GOTS
1,BRAND-0002,Brand_2,Japan,2015,D,Vegan Leather,True,124.39,1951566.31,37267.75,False,15,250.07,Growing,GOTS
2,BRAND-0003,Brand_3,USA,2024,A,Vegan Leather,False,336.66,467454.52,38385.92,False,2,146.16,Growing,B Corp
3,BRAND-0004,Brand_4,Italy,2023,D,Bamboo Fabric,False,152.04,899576.90,32665.45,False,13,165.52,Stable,OEKO-TEX
4,BRAND-0005,Brand_5,USA,2016,D,Bamboo Fabric,True,415.63,1809219.90,37295.47,True,19,211.63,Stable,Fair Trade


Membuat expectation suite untuk menyimpan hasil validasi dan membuat validator untuk proses validasi

## 4.1. To be Unique

In [8]:
# Memvalidasi Value Column harus Unique

validator.expect_column_values_to_be_unique('brand_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "result": {
    "element_count": 5000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Validasi kolom `brand_id` dengan value harus unique berhasil. kolom ini merupakan index dari data yang dipakai oleh karena itu, kolom ini harus berisikan Unique Value

## 4.2. To be Between min_value and max_value

In [11]:
# Validasi value minimum dan maximum

validator.expect_column_values_to_be_between(
    column='average_price_usd', min_value=0, max_value=500
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "result": {
    "element_count": 5000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Validasi kolom `average_price_usd` dengan minimum dan maximum value yang harus dipenuhi menghasilkan output sukses. Kolom tersebut merupakan kolom yang berisi harga rata-rata sebuah produk sehingga jika terdapat nilai yang extreme akan merusak data dan tidak masuk akal secara praktis

## 4.3. To be in Set

In [12]:
# Validasi Value Kolom harus berisikan set list yang ditentukan

validator.expect_column_values_to_be_in_set('sustainability_rating', ['A','B','C','D'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "result": {
    "element_count": 5000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Validasi kolom `sustainability_rating` dengan syarat value berisikan `A`,`B`,`C`, atau `D` menghasilkan output Sukses. Kolom ini merupakan rating produk terhadap tingkat sustainability-nya

## 4.4. To be in Type List

In [28]:
# Validasi value kolom harus mengikuti tipe data pada list

validator.expect_column_values_to_be_in_type_list('recycling_programs', ['object','bool'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "result": {
    "observed_value": "bool_"
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Validasi kolom `recycling_programs` dengan syarat value bertipe data `object` atau `bool` menghasilkan output sukses. Kolom ini merupakan status program daur ulang pada sebuah pabrik manufaktur sehingga value hanya berisi `True/False` atau `Yes/No`

## 4.5. Table Column count to Equal

In [16]:
# Validasi total kolom harus sama dengan yang ditentukan
validator.expect_table_column_count_to_equal(15)

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "result": {
    "observed_value": 15
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Validasi jumlah kolom harus berjumlah 15 menghasilkan output sukses. Data ini merupakan data terstruktur sehingga data yang masuk harus mengikuti aturan-aturan yang ditentukan

## 4.6. Compound Columns to be Unique

In [18]:
# Validasi beberapa kolom harus memiliki value yang berbeda satu sama lain
validator.expect_compound_columns_to_be_unique(column_list=['carbon_footprint_mt','water_usage_liters', 'waste_production_kg'])

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "result": {
    "element_count": 5000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Validasi kolom `carbon_footprint_mt`,`water_usage_liters`, dan `waste_production_kg` dengan syarat harus memiliki value yang berbeda tiap kolomnya menghasilkan output sukses. Ketiga kolom ini merupakan indikator besaran pemakaian air dan pembuangan limbah sehingga ketiga kolom ini pasti memiliki angka yang berbeda pada realitanya.

## 4.7. Table Columns to not Match

In [30]:
# Validasi value kolom tidak boleh sama
validator.expect_table_columns_to_match_set(column_set=['product_lines','average_price_usd'], exact_match=False)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "meta": {},
  "result": {
    "observed_value": [
      "average_price_usd",
      "brand_id",
      "brand_name",
      "carbon_footprint_mt",
      "certifications",
      "country",
      "eco_friendly_manufacturing",
      "market_trend",
      "material_type",
      "product_lines",
      "recycling_programs",
      "sustainability_rating",
      "waste_production_kg",
      "water_usage_liters",
      "year"
    ],
    "details": {
      "mismatched": {
        "unexpected": [
          "brand_id",
          "brand_name",
          "carbon_footprint_mt",
          "certifications",
          "country",
          "eco_friendly_manufacturing",
          "market_trend",
          "material_type",
          "recycling_programs",
          "sustainability_rating",
          "waste_production_kg",
          "water_usage_liters",
          "year"
        ]
      }
    }
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    

Validasi kolom `product_lines` dan `average_price_usd` dengan syarat memiliki value yang tidak sama. Kedua kolom ini merupakan kolom yang memiliki angka namun nilainya memiliki perbedaan yang signifikan, validasi ini bertujuan untuk cross-check pengisian data oleh surveyor

# 5. Save

In [ ]:
# Save into Expectation Suite

validator.save_expectation_suite()

Save validator yang sukses ke dalam expectation suite